In [2]:
import getpass
import os
os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API Key:")

In [4]:
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai.chat_models import ChatOpenAI

tools = [TavilySearchResults(max_results=1)]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
# llm = ChatOpenAI(model="gpt-3.5-turbo-1106", streaming=True)
llm = ChatOpenAI(
    model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
    base_url="http://10.2.125.37:1234/v1",
    api_key="lm-studio"
) 

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent_runnable, tools=tools)


agent_executor.invoke({"input": "what is the weather in sf", "chat_history": []})

{'input': 'what is the weather in sf',
 'chat_history': [],
 'output': "San Francisco's weather can be quite cool and foggy, especially near the coast. As I'm a text-based AI, I don't have real-time access to current weather conditions. However, I can suggest some ways for you to check the current weather in San Francisco:\n\n1. **Check online weather websites**: You can visit websites like AccuWeather, Weather.com, or the National Weather Service (NWS) for up-to-date information on San Francisco's weather.\n2. **Use a mobile app**: Download a weather app on your smartphone, such as Dark Sky or Weather Underground, to get real-time weather updates for San Francisco.\n\nThat being said, I can give you some general information about San Francisco's climate:\n\n* **Temperature:** Daytime temperatures in San Francisco typically range from the mid-60s to low 70s Fahrenheit (18-22°C) during the summer months and around 50°F (10°C) during the winter months.\n* **Fog:** San Francisco is known 

In [5]:
import operator
from typing import Annotated, TypedDict, Union

from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    # The input string
    input: str
    # The list of previous messages in the conversation
    chat_history: list[BaseMessage]
    # The outcome of a given call to the agent
    # Needs `None` as a valid type, since this is what this will start as
    agent_outcome: Union[AgentAction, AgentFinish, None]
    # List of actions and corresponding observations
    # Here we annotate this with `operator.add` to indicate that operations to
    # this state should be ADDED to the existing values (not overwrite it)
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

In [7]:
from langchain_core.agents import AgentFinish

from langgraph.prebuilt.tool_executor import ToolExecutor

# This a helper class we have that is useful for running tools
# It takes in an agent action and calls that tool and returns the result
tool_executor = ToolExecutor(tools)


# Define the agent
def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}


# Define the function to execute tools
def execute_tools(data):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data["agent_outcome"]
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}


# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

In [8]:
from langgraph.graph import END, StateGraph, START

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [9]:
inputs = {"input": "what is the weather in sf", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentFinish(return_values={'output': "However, I'm a large language model, I don't have real-time access to current weather conditions. But I can suggest some ways for you to find out the current weather in San Francisco.\n\nYou can:\n\n1. Check online weather websites such as:\n\t* AccuWeather: accuweather.com\n\t* Weather.com: weather.com\n\t* National Weather Service (NWS): weather.gov\n2. Use a mobile app like Dark Sky or Weather Underground to get real-time weather updates.\n3. Tune into local news stations or check their websites for the latest weather forecast.\n\nAs of my knowledge cutoff, San Francisco's weather is typically mild and cool year-round, with some foggy days in the summer and chilly days in the winter. But please note that this information may not reflect the current conditions.\n\nWould you like to know more about San Francisco's climate or average temperature ranges?"}, log="However, I'm a large language model, I don't have real-time access to 